In [1]:
import numpy as np
import pandas as pd

from scipy.stats import multivariate_normal, norm, bernoulli

import pymc3 as pm

In [2]:
N = 100

h0 = norm.rvs(size=N, loc=10, scale=2)

treatment = np.array([0,1]*int(N/2))
fungus    = bernoulli.rvs(size=N, p=0.5-treatment*0.4)
h1        = h0 + norm.rvs(size=N, loc=5-3*fungus)

### Create a testing sample

In [3]:
N_test = 100

h0_test = norm.rvs(size=N_test, loc=10, scale=2)

treatment_test = np.array([0,1]*int(N_test/2))
fungus_test    = bernoulli.rvs(size=N_test, p=0.5-treatment_test*0.4)
h1_test        = h0_test + norm.rvs(size=N_test, loc=5-3*fungus_test)

## Confounded Model

In [4]:
with pm.Model() as model_confounded:
    alpha  = pm.Lognormal('alpha', mu=10, sigma=100)
    beta_T = pm.Normal('beta_T', mu=0, sigma=0.5)
    beta_F = pm.Normal('beta_F', mu=0, sigma=0.5)
    sigma  = pm.Exponential('sigma', lam=1)
    p      = alpha + beta_T*treatment + beta_F*fungus
    mu     = h0*p
    h      = pm.Normal('height', mu=mu, sd=sigma, observed=h1)
    
    par_post = pm.find_MAP()
    hessian  = pm.find_hessian(par_post, vars=[alpha, beta_T, beta_F, sigma])

In [5]:
param_mu_post = [par_post['alpha'], par_post['beta_T'], par_post['beta_F'], par_post['sigma']]
sample_post   = multivariate_normal.rvs(size=10**4, mean=param_mu_post, cov=np.linalg.inv(hessian))
sample_post   = pd.DataFrame(sample_post, columns=['alpha', 'beta_T', 'beta_F', 'sigma'])

### Calculate the WAIC

In [8]:
lppd = 0
penalty = 0
WAIC_sample = []

for i in range(N):
    p_i = sample_post['alpha']+sample_post['beta_T']*treatment[i]+sample_post['beta_F']*fungus[i]
    p_y_post = norm.pdf(
        h1[i],
        loc = h0[i]*p_i,
        scale = sample_post['sigma']
    )

    p_y_post         = p_y_post[~np.isnan(p_y_post)]
    log_p_y_post     = np.log(np.mean(p_y_post))
    p_y_post         = p_y_post[p_y_post>0]
    var_log_p_y_post = np.var(np.log(p_y_post)) 
    lppd    += log_p_y_post
    penalty += var_log_p_y_post
    WAIC_sample.append(-2*(log_p_y_post-var_log_p_y_post))

WAIC = -2*(lppd - penalty)
WAIC_std_error = np.sqrt(N*np.var(WAIC_sample))

In [11]:
WAIC, WAIC_std_error

(357.7932458475005, 15.994192641747137)

### Calculate the out-of-sample error, i.e. the error on the testing sample

In [13]:
deviance_test = 0
deviance_test_sample = []

for i in range(N_test):
    p_i = sample_post['alpha']+sample_post['beta_T']*treatment[i]+sample_post['beta_F']*fungus[i]
    p_y_post = norm.pdf(
        h1[i],
        loc = h0[i]*p_i,
        scale = sample_post['sigma']
    )

    p_y_post         = p_y_post[~np.isnan(p_y_post)]
    log_p_y_post     = np.log(np.mean(p_y_post))
    p_y_post         = p_y_post[p_y_post>0]
    var_log_p_y_post = np.var(np.log(p_y_post)) 
    deviance_test    += -2*log_p_y_post
    deviance_test_sample.append(-2*log_p_y_post)

In [17]:
deviance_test, np.sqrt(N_test*np.var(deviance_test_sample))

(350.10664139759933, 14.628168434118683)

### Calculating the CV error

In [34]:
deviance_cv = 0
deviance_cv_sample = []

for i in range(N):
    
    with pm.Model() as model_confounded:
        alpha  = pm.Lognormal('alpha', mu=10, sigma=100)
        beta_T = pm.Normal('beta_T', mu=0, sigma=0.5)
        beta_F = pm.Normal('beta_F', mu=0, sigma=0.5)
        sigma  = pm.Exponential('sigma', lam=1)
        p      = alpha + beta_T*np.delete(treatment,i) + beta_F*np.delete(fungus,i)
        mu     = np.delete(h0,i)*p
        h      = pm.Normal('height', mu=mu, sd=sigma, observed=np.delete(h1,i))
        
        par_post = pm.find_MAP()
        hessian  = pm.find_hessian(par_post, vars=[alpha, beta_T, beta_F, sigma])

    param_mu_post = [par_post['alpha'], par_post['beta_T'], par_post['beta_F'], par_post['sigma']]
    sample_post   = multivariate_normal.rvs(size=10**4, mean=param_mu_post, cov=np.linalg.inv(hessian))
    sample_post   = pd.DataFrame(sample_post, columns=['alpha', 'beta_T', 'beta_F', 'sigma'])

    p_i = sample_post['alpha']+sample_post['beta_T']*treatment[i]+sample_post['beta_F']*fungus[i]
    p_y_post = norm.pdf(
        h1[i],
        loc = h0[i]*p_i,
        scale = sample_post['sigma']
    )

    p_y_post         = p_y_post[~np.isnan(p_y_post)]
    log_p_y_post     = np.log(np.mean(p_y_post))
    p_y_post         = p_y_post[p_y_post>0]
    deviance_cv     += -2*log_p_y_post
    deviance_cv_sample.append(-2*log_p_y_post)
    

In [39]:
deviance_cv, np.sqrt(N*np.var(deviance_cv_sample))

(357.6256371795438, 15.95589605443246)

## Unconfounded Model

In [40]:
with pm.Model() as model_confounded:
    alpha  = pm.Lognormal('alpha', mu=10, sigma=100)
    beta_T = pm.Normal('beta_T', mu=0, sigma=0.5)
    sigma  = pm.Exponential('sigma', lam=1)
    p      = alpha + beta_T*treatment
    mu     = h0*p
    h      = pm.Normal('height', mu=mu, sd=sigma, observed=h1)
    
    par_post = pm.find_MAP()
    hessian  = pm.find_hessian(par_post, vars=[alpha, beta_T, sigma])

In [41]:
param_mu_post = [par_post['alpha'], par_post['beta_T'], par_post['sigma']]
sample_post   = multivariate_normal.rvs(size=10**4, mean=param_mu_post, cov=np.linalg.inv(hessian))
sample_post   = pd.DataFrame(sample_post, columns=['alpha', 'beta_T', 'sigma'])

### Calculate the WAIC

In [42]:
lppd = 0
penalty = 0
WAIC_sample = []

for i in range(N):
    p_i = sample_post['alpha']+sample_post['beta_T']*treatment[i]
    p_y_post = norm.pdf(
        h1[i],
        loc = h0[i]*p_i,
        scale = sample_post['sigma']
    )

    p_y_post         = p_y_post[~np.isnan(p_y_post)]
    log_p_y_post     = np.log(np.mean(p_y_post))
    p_y_post         = p_y_post[p_y_post>0]
    var_log_p_y_post = np.var(np.log(p_y_post)) 
    lppd    += log_p_y_post
    penalty += var_log_p_y_post
    WAIC_sample.append(-2*(log_p_y_post-var_log_p_y_post))

WAIC = -2*(lppd - penalty)
WAIC_std_error = np.sqrt(N*np.var(WAIC_sample))

In [43]:
WAIC, WAIC_std_error

(404.6404432754099, 13.213863571420891)

### Calculate the out-of-sample error, i.e. the error on the testing sample

In [47]:
deviance_test = 0
deviance_test_sample = []

for i in range(N_test):
    p_i = sample_post['alpha']+sample_post['beta_T']*treatment[i]
    p_y_post = norm.pdf(
        h1[i],
        loc = h0[i]*p_i,
        scale = sample_post['sigma']
    )

    p_y_post         = p_y_post[~np.isnan(p_y_post)]
    log_p_y_post     = np.log(np.mean(p_y_post))
    p_y_post         = p_y_post[p_y_post>0]
    var_log_p_y_post = np.var(np.log(p_y_post)) 
    deviance_test    += -2*log_p_y_post
    deviance_test_sample.append(-2*log_p_y_post)

In [48]:
deviance_test, np.sqrt(N_test*np.var(deviance_test_sample))

(398.99849933888663, 12.272473108599478)